In [1]:
import garpar as gp
import itertools as it
import pandas as pd
import numpy as np
import joblib as jb
import tqdm.notebook
import scipy 
import matplotlib.pyplot as plt
import seaborn as sns
from lib import lsboostrap

In [2]:
merval21 = gp.datasets.load_MERVAL(
    first="01-02-2021", last = "12-31-2021")
merval21 = merval21.as_returns()
merval21

Stocks,ALUA,BBAR,BYMA,CEPU,COME,CRES,CVH,EDN,GGAL,HARG,...,MOLA,MOLI,MORI,OEST,PATA,POLL,RIGO,SAMI,SEMI,TGLT
Days,,,,,,,,,,,,,,,,,,,,,
2021-01-05,0.056995,-0.014122,-0.003331,0.039242,0.004132,0.027451,0.020047,-0.012456,0.003245,0.016598,...,0.028148,0.006873,0.046358,-0.001789,0.021505,0.076503,0.000000,0.000000,-0.010889,-0.006061
2021-01-06,0.035294,-0.012325,0.008354,-0.026042,0.008230,0.013740,0.035838,-0.007207,-0.004852,0.016327,...,0.011527,-0.010239,-0.006329,-0.017921,-0.026316,0.000000,0.015385,0.008475,0.023853,0.006098
2021-01-07,0.005682,-0.015514,-0.000829,-0.005348,-0.004082,0.038404,0.001116,-0.014519,0.006908,-0.006024,...,0.001425,-0.006897,0.054140,0.000000,0.000000,0.000000,0.030303,0.000000,0.109319,-0.012121
2021-01-08,0.005650,-0.002398,-0.000829,-0.013441,-0.012295,-0.015228,-0.125975,-0.009208,-0.010089,-0.004040,...,-0.001422,0.005787,-0.012085,-0.023723,-0.005405,0.000000,-0.011029,0.008403,-0.046850,-0.053170
2021-01-11,0.007491,-0.050137,-0.004149,-0.016349,-0.004149,0.002946,-0.019133,-0.040892,-0.019568,-0.012170,...,-0.027066,-0.016110,-0.024465,-0.009346,0.000000,-0.005076,-0.011152,0.008333,0.033898,0.010799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,0.024202,0.020322,0.031205,0.029366,0.001681,0.000996,0.021193,0.024510,0.003778,0.004280,...,0.016570,0.012295,-0.024922,0.048493,0.056319,0.000000,-0.006579,-0.001786,0.005025,0.008021
2021-12-27,-0.008593,0.019689,-0.017691,-0.010511,0.028523,-0.056716,-0.039969,0.029665,0.004266,-0.036932,...,0.029956,0.000000,0.012780,0.018750,-0.016905,-0.024299,0.000000,0.016100,0.000000,-0.013263
2021-12-28,-0.050921,-0.049394,-0.032227,-0.042489,0.001631,0.007911,-0.013611,-0.036245,-0.034733,-0.022124,...,0.028229,0.004049,0.000000,-0.019632,-0.010582,-0.007663,0.000000,-0.019366,0.001667,0.008065


In [3]:
# definimos los sectores 
sectores = {'come': ["AUSO", "BOLT", "DYCA","GCLA", "IRCP", "IRSA", "OEST", "PATA", "POLL","TECO2","CVH"], 
'moa': ["CELU", "GRIM", "HAVA", "LEDE", "MOLA", "MOLI", "MORI", "SAMI", "SEMI"],
'moi': ["AGRO", "ALUA",  "FERR", "LONG", "RIGO", "HARG", "FIPL"],
'agro': ["CADO", "GARO", "INVJ","CRES"], 
'ene': ["CEPU", "CGPA2", "DGCU2", "GBAN", "METR", "TGNO4", "TGSU2", "YPFD", "PAMP","TRAN","COME","EDN","GBAN" ], 
'fin': ["BHIP", "BPAT",  "BYMA", "GGAL", "SUPV", "BBAR"] }

In [4]:
for sector,stocks in sectores.items():
    stocks = stocks
    result= lsboostrap.levy_stable(
    df=merval21, frac=0.8, columns=stocks, iterations=1000, rstate=1, n_jobs=50)
    filename = f"2021_{sector}.jpkl"
    jb.dump(result,filename)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:47<00:00, 23.66s/it]


In [6]:
merval20 = gp.datasets.load_MERVAL(
    first="01-02-2020", last = "12-31-2020")
merval20 = merval20.as_returns()
merval20

Stocks,ALUA,BBAR,BYMA,CEPU,COME,CRES,CVH,EDN,GGAL,HARG,...,MOLA,MOLI,MORI,OEST,PATA,POLL,RIGO,SAMI,SEMI,TGLT
Days,,,,,,,,,,,,,,,,,,,,,
2020-01-03,-0.056250,-0.032014,-0.006319,-0.064935,-0.039823,-0.016832,-0.013910,-0.041850,-0.017886,-0.011340,...,-0.008547,-0.004934,-0.006667,-0.001692,0.000000,0.027027,-0.035088,-0.017241,-0.025157,0.084906
2020-01-06,0.023179,0.046823,-0.014308,0.047840,0.036866,0.020141,0.017241,0.055172,0.033940,-0.008342,...,-0.008621,-0.016529,-0.015660,-0.011864,0.019608,0.000000,-0.009091,-0.008772,-0.003226,-0.034783
2020-01-07,-0.006472,0.009585,0.000000,-0.008837,-0.022222,0.024679,-0.036980,-0.008715,-0.007206,0.008412,...,-0.002899,-0.033613,0.034091,0.008576,0.055769,0.000000,0.045872,-0.017699,-0.016181,-0.045045
2020-01-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-09,-0.040717,0.059072,-0.024194,0.043091,0.009091,-0.045279,0.024000,0.008791,0.051210,-0.029197,...,-0.002907,0.026087,0.252747,-0.013605,0.129326,0.000000,0.070175,-0.018018,0.101974,-0.024528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-22,0.034921,0.030345,0.015833,-0.008219,0.008197,0.022663,0.007075,0.002012,0.008358,0.026104,...,-0.015942,0.000000,0.000000,0.010471,-0.029178,0.000000,0.000000,-0.010593,-0.007353,0.007905
2020-12-23,-0.009202,0.043670,0.023790,0.035912,0.032520,0.030471,-0.007026,0.034137,0.041028,-0.007828,...,-0.007364,0.000000,0.003257,0.018998,0.035519,0.000000,0.000000,0.027837,0.016667,0.000000
2020-12-28,-0.030960,-0.023354,-0.022436,-0.034667,-0.011811,0.044803,-0.036557,0.011650,-0.002389,-0.005917,...,-0.020030,0.000000,-0.019481,-0.023729,-0.013193,-0.052632,0.028037,-0.025000,-0.007286,0.000000


In [7]:
for sector,stocks in sectores.items():
    stocks = stocks
    result= lsboostrap.levy_stable(
    df=merval20, frac=0.8, columns=stocks, iterations=2, rstate=1, n_jobs=1)
    filename = f"2020_{sector}.csv"
    jb.dump(result,filename)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:19<00:00, 39.96s/it]


In [10]:
merval19 = gp.datasets.load_MERVAL(
    first="01-02-2019", last = "12-31-2019")
merval19 = merval19.as_returns()
merval19

Stocks,ALUA,BBAR,BYMA,CEPU,COME,CRES,CVH,EDN,GGAL,HARG,...,MOLA,MOLI,MORI,OEST,PATA,POLL,RIGO,SAMI,SEMI,TGLT
Days,,,,,,,,,,,,,,,,,,,,,
2019-01-03,0.014493,0.002139,-0.015258,-0.005806,-0.026471,-0.005435,-0.012762,-0.042471,0.008883,0.000000,...,-0.016509,-0.021053,-0.015789,0.000000,0.000000,0.000000,-0.027473,-0.025907,-0.003390,0.000000
2019-01-04,0.005714,0.052295,-0.002605,0.016058,0.045317,0.018579,0.052961,0.019153,0.056070,-0.004237,...,0.103118,0.073925,0.026738,0.035714,0.000000,-0.084211,0.016949,0.053191,0.017007,0.048714
2019-01-07,0.039773,0.027721,0.024196,-0.008621,0.011561,0.033262,-0.009901,-0.002967,0.031154,-0.002553,...,0.000000,0.001252,0.000000,0.004138,-0.006198,-0.022989,0.000000,0.009091,-0.003344,0.006452
2019-01-08,0.021858,0.006579,-0.017450,-0.008696,0.002857,-0.008307,-0.017000,0.001984,-0.008511,-0.006826,...,0.000000,0.006250,-0.010417,0.010989,-0.002079,0.000000,0.000000,-0.019019,0.013423,0.025641
2019-01-09,0.018717,0.022876,0.009563,0.064327,0.000000,0.019895,0.057986,0.021782,0.034335,0.013746,...,0.004348,-0.018634,0.010526,0.013587,0.000000,0.000000,-0.011111,-0.005102,0.000000,-0.025000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-20,-0.009390,0.009346,-0.001672,0.041237,-0.005291,0.060280,-0.039400,-0.004785,0.009651,-0.003268,...,0.009763,0.028846,-0.012876,-0.019678,0.022222,0.000000,0.000000,0.045045,-0.021875,0.004376
2019-12-23,0.001580,0.069646,0.018425,0.008487,0.010638,0.036548,0.000000,0.057692,0.069185,0.005464,...,-0.033149,0.018692,-0.021739,-0.014599,0.000000,0.000000,0.004464,0.008621,0.000000,-0.017429
2019-12-26,0.025237,0.117802,0.074013,0.107994,0.042105,0.077375,0.089844,0.218182,0.068540,0.047826,...,0.022857,0.082569,-0.015556,0.055556,0.065217,0.010989,0.113333,0.004274,0.012780,0.019956


In [11]:
for sector,stocks in sectores.items():
    stocks = stocks
    result= lsboostrap.levy_stable(
    df=merval19, frac=0.8, columns=stocks, iterations=2, rstate=1, n_jobs=1)
    filename = f"2019_{sector}.csv"
    jb.dump(result,filename)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:25<00:00, 42.66s/it]


In [13]:
merval18 = gp.datasets.load_MERVAL(
    first="01-02-2018", last = "12-31-2018")
merval18 = merval18.as_returns()
merval18

Stocks,ALUA,BBAR,BYMA,CEPU,COME,CRES,CVH,EDN,GGAL,HARG,...,MOLA,MOLI,MORI,OEST,PATA,POLL,RIGO,SAMI,SEMI,TGLT
Days,,,,,,,,,,,,,,,,,,,,,
2018-01-03,0.035000,-0.012092,-0.041402,0.043633,-0.008114,0.009861,-0.002079,0.004292,-0.007232,0.005664,...,-0.005810,-0.013115,-0.042254,-0.019286,0.000000,0.019251,-0.015817,0.028211,-0.008571,0.028249
2018-01-04,0.002761,0.026042,-0.025132,0.062287,-0.006135,0.025038,0.020729,0.011752,0.051396,-0.011137,...,-0.005612,0.001661,0.000000,0.012621,-0.035714,0.007345,-0.008117,0.021294,0.020173,0.016484
2018-01-05,-0.011700,0.015990,0.006020,0.018474,-0.002058,-0.009282,-0.008878,-0.002112,-0.001155,0.012557,...,0.002851,0.000000,0.029412,0.000000,0.000000,0.031250,-0.008020,0.046512,-0.005650,-0.010811
2018-01-08,-0.021588,0.011429,-0.004296,-0.007624,0.000000,-0.003698,-0.005354,0.022222,-0.014258,0.051521,...,-0.019611,-0.029851,0.014286,0.009855,0.000000,-0.039394,0.007260,0.024904,-0.025568,-0.016393
2018-01-09,-0.009964,-0.011300,-0.019572,0.005033,-0.016495,-0.011631,-0.006211,-0.005176,-0.008600,0.043161,...,-0.017163,0.006838,-0.051643,-0.002870,0.016667,-0.001052,-0.023587,0.000000,0.002915,-0.027778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-20,-0.019178,-0.023396,-0.009620,-0.013413,-0.002967,-0.031625,-0.017366,-0.032161,-0.019006,-0.020212,...,-0.004237,0.018767,-0.037037,-0.027982,-0.007937,0.000000,0.000000,-0.032427,-0.019672,-0.006667
2018-12-21,-0.016760,-0.052995,0.002943,-0.012085,-0.032738,-0.050676,-0.020619,-0.017653,-0.041729,-0.017682,...,-0.106383,-0.013158,-0.016484,-0.030303,-0.020000,0.000000,0.000000,-0.016216,-0.016722,-0.026846
2018-12-26,0.000000,-0.002300,0.001908,-0.012232,-0.012308,0.013049,0.025994,0.015856,-0.009850,0.006000,...,0.014286,-0.020000,0.005587,0.060938,0.000000,0.000000,0.003333,-0.016484,0.003401,-0.034483


In [14]:
for sector,stocks in sectores.items():
    stocks = stocks
    result= lsboostrap.levy_stable(
    df=merval18, frac=0.8, columns=stocks, iterations=2, rstate=1, n_jobs=1)
    filename = f"2018_{sector}.csv"
    jb.dump(result,filename)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:19<00:00, 39.78s/it]
